In [3]:
import string 
import random

In [2]:
cc = string.ascii_lowercase

In [59]:
def name():
    return 'X_'+ ''.join(random.choices(cc,k=3)) + '.html'

In [16]:
name()

'X_dis.md'

In [50]:
import numpy as np

n = 7
M = np.identity(n,dtype =np.int)
# this slides everything over by 1
M = np.roll(M,1, axis=1)

M = M + np.random.randint(2,size=(n,n))
M[M>1] = 1


In [51]:
M

array([[0, 1, 1, 1, 0, 1, 0],
       [1, 1, 1, 1, 1, 0, 1],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 0, 1, 0, 1],
       [0, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 1, 0, 0, 1],
       [1, 0, 1, 1, 1, 0, 0]])

In [60]:
    

sites = [name() for k in range(n)]
lks = dict(zip(sites,M))


In [61]:
for x, ll in lks.items():
    lks[x] = [ x for x,y in zip(sites,ll) if y >0]

In [62]:
lks

{'X_lzx.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_dbo.html'],
 'X_ask.html': ['X_lzx.html',
  'X_ask.html',
  'X_hvc.html',
  'X_lnh.html',
  'X_cgg.html',
  'X_ifl.html'],
 'X_hvc.html': ['X_hvc.html', 'X_lnh.html', 'X_cgg.html', 'X_dbo.html'],
 'X_lnh.html': ['X_hvc.html', 'X_cgg.html', 'X_ifl.html'],
 'X_cgg.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_dbo.html'],
 'X_dbo.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_ifl.html'],
 'X_ifl.html': ['X_lzx.html', 'X_hvc.html', 'X_lnh.html', 'X_cgg.html']}

In [66]:
for x, ll in lks.items():
    with open(x,'w') as fp:
        
        fp.write('# ' + x + '<br>\n'*3 )
        body = ''.join(['<a href="./{}"> {}</a> <br>\n'.format(y,y) for  y in ll])
        fp.write(body)

In [67]:
! ../.g

[master 29fbea1] web
 15 files changed, 108 insertions(+), 70 deletions(-)
 delete mode 100644 PROG/X_aks.md
 create mode 100644 PROG/X_ask.html
 create mode 100644 PROG/X_cgg.html
 delete mode 100644 PROG/X_ckk.md
 create mode 100644 PROG/X_dbo.html
 delete mode 100644 PROG/X_hdu.md
 create mode 100644 PROG/X_hvc.html
 create mode 100644 PROG/X_ifl.html
 create mode 100644 PROG/X_lnh.html
 delete mode 100644 PROG/X_lrl.md
 create mode 100644 PROG/X_lzx.html
 delete mode 100644 PROG/X_srj.md
 delete mode 100644 PROG/X_xcg.md
 delete mode 100644 PROG/X_ynh.md
Counting objects: 11, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (11/11), done.
Writing objects: 100% (11/11), 1.61 KiB | 822.00 KiB/s, done.
Total 11 (delta 9), reused 0 (delta 0)
remote: Resolving deltas: 100% (9/9), completed with 3 local objects.
To https://github.com/macbuse/macbuse.github.io.git
   cc93672..29fbea1  master -> master
